In [1]:
import sys
!"{sys.executable}" -m pip install feedparser
!"{sys.executable}" -m pip install markdown2
!"{sys.executable}" -m pip install langchain_community
!"{sys.executable}" -m pip install langchain_ollama

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Arxiv retriever

1) Write python function to retrieve all ids, categories and abstracts within the last day
2) Parse information and store in df
3) Group information based on categories
4) Summarize infromation with LLMs
5) Send information via Email
6) Automate task via CRON job
7) Podcast???

In [9]:
from datetime import datetime, timedelta
import pandas as pd
from langchain_ollama import OllamaLLM
import feedparser
import urllib, urllib.request
import os
import requests

def get_dates():
    last_day = datetime.today() - timedelta(days=2)
    today = datetime.today()
    return last_day.strftime('%Y%m%d0000'), today.strftime('%Y%m%d0000')

In [10]:
def fetch_data():
    max_res = 2000
    start_date, end_date = get_dates()
    url = f'http://export.arxiv.org/api/query?search_query=submittedDate:[{start_date}+TO+{end_date}]&sortBy=submittedDate&sortOrder=ascending&start=0&max_results={max_res}'
    print(url)
    data = urllib.request.urlopen(url)
    xml = data.read().decode('utf-8')
    return xml

In [11]:
def get_prop_array(parsed_xml, prop):
    entries = parsed_xml["entries"]
    prop_array = list(map(lambda e: e[prop], entries))
    return prop_array

def parse_parsed_xml_to_df(parsed_xml):
    ids = get_prop_array(parsed_xml, "id")
    links = get_prop_array(parsed_xml, "link")
    summaries = get_prop_array(parsed_xml, "summary")
    authors = get_prop_array(parsed_xml, "authors")
    authors = list(map(lambda author_array: ",".join([a["name"] for a in author_array]), authors))
    tags = get_prop_array(parsed_xml, "tags")
    tags = list(map(lambda t_arr: ",".join([t["term"] for t in t_arr]), tags))
    primary_cat = get_prop_array(parsed_xml, "arxiv_primary_category")
    primary_cat = list(map(lambda c: c["term"], primary_cat))
    df_data = {
    "ids": ids,
    "links": links,
    "summaries": summaries,
    "authors": authors,
    "tags":tags,
    "primary_cat": primary_cat
    }
    return pd.DataFrame(df_data)
    
def store_xml_as_df():
    retrieved_xml = fetch_data()
    parsed_xml = feedparser.parse(retrieved_xml)
    df = parse_parsed_xml_to_df(parsed_xml)
    os.makedirs("papers", exist_ok=True)
    df.to_csv(f"papers/{datetime.today().strftime('%Y%m%d0000')}.csv", index = False)
    return df

In [12]:
store_xml_as_df()

http://export.arxiv.org/api/query?search_query=submittedDate:[202505150000+TO+202505170000]&sortBy=submittedDate&sortOrder=ascending&start=0&max_results=2000


In [13]:
data_df = pd.read_csv("papers/202505170000.csv")

In [14]:
data_df

,ids,links,summaries,authors,tags,primary_cat
0,http://arxiv.org/abs/2505.09864v1,http://arxiv.org/abs/2505.09864v1,"Over the past decade, the use of machine learn...",Aditya Panangat,cs.LG,cs.LG
1,http://arxiv.org/abs/2505.09865v1,http://arxiv.org/abs/2505.09865v1,"The circular $\beta$ ensemble for $\beta =1,2$...","Peter J. Forrester,Bo-Jian Shen","math-ph,math.MP",math-ph
2,http://arxiv.org/abs/2505.09866v1,http://arxiv.org/abs/2505.09866v1,"In this paper, we investigate quasinormal mode...","Luiz F. Ferreira,Alfredo Vega","hep-ph,hep-th",hep-ph
3,http://arxiv.org/abs/2505.09867v1,http://arxiv.org/abs/2505.09867v1,This study examines stress levels in roadway w...,"Fatemeh Banani Ardecani,Omidreza Shoghli",cs.HC,cs.HC
4,http://arxiv.org/abs/2505.09868v1,http://arxiv.org/abs/2505.09868v1,"Despite its U.S. constitutional foundation, th...","Tin Trung Nguyen,Jiannan Xu,Phuong-Anh Nguyen-...","cs.CY,cs.AI,cs.HC",cs.CY
...,...,...,...,...,...,...
698,http://arxiv.org/abs/2505.10562v1,http://arxiv.org/abs/2505.10562v1,Existing vision tokenization isolates the opti...,"Wenxuan Wang,Fan Zhang,Yufeng Cui,Haiwen Diao,...",cs.CV,cs.CV
699,http://arxiv.org/abs/2505.10563v1,http://arxiv.org/abs/2505.10563v1,The halide double perovskite Cs$_2$AgBiBr$_6$ ...,"Collin Tower,Fereidoon S. Razavi,Jeremy Dion,J...",cond-mat.mtrl-sci,cond-mat.mtrl-sci
700,http://arxiv.org/abs/2505.10564v1,http://arxiv.org/abs/2505.10564v1,Studies of young planets help us understand pl...,"J. -F. Donati,E. Gaidos,C. Moutou,P. I. Cristo...","astro-ph.SR,astro-ph.EP",astro-ph.SR
701,http://arxiv.org/abs/2505.10565v1,http://arxiv.org/abs/2505.10565v1,"This work presents Prior Depth Anything, a fra...","Zehan Wang,Siyu Chen,Lihe Yang,Jialei Wang,Zia...",cs.CV,cs.CV


In [15]:
def get_prompt(df_string):
    return f"""You are an english AI assistant. Your primary task is to generate a consolidated english summary in **raw well formatted Markdown format**. This summary will be based on a list of research contributions that will be provided to you. Each contribution includes the author(s), a brief summary of their work, and a direct HTML link for further details.
    
    The main goal is to synthesize these individual english summaries into a single, cohesive narrative. This narrative should clearly highlight what the authors have accomplished or discovered. The final output must be *only* the markdown summary itself, without any of your own conversational text before or after it.
    
    **Input Data Format:**
    
    You will receive the data as a single block of text. Within this block, each research entry will be distinctly separated from the next. Each entry will adhere to the following structure:
    
    Authors: [Full Names of Authors]
    Summary: [A concise summary of the work done]
    Link: [A full HTML URL to the source or more information]
    --- (This line with three hyphens acts as a separator between individual entries)
    
    **Example of Input Data Block (Illustrative):**
    
    Authors: Dr. Ada Lovelace, Charles Babbage
    Summary: They conceptualized the Analytical Engine, a groundbreaking mechanical general-purpose computer, detailing its potential for complex calculations and even for composing music.
    Link: https://example.com/analytical_engine_details
    ---
    Authors: Dr. Grace Hopper
    Summary: A pioneer in computer programming, Dr. Hopper developed the first compiler (A-0 System) and was instrumental in the development of COBOL. She also popularized the term "debugging" after a moth was found in a relay.
    Link: https://example.com/grace_hopper_compiler_work
    ---
    Authors: Dr. Tim Berners-Lee
    Summary: He is credited with inventing the World Wide Web. His work included developing the first web browser, web server, HTTP (Hypertext Transfer Protocol), and HTML (Hypertext Markup Language), fundamentally changing how information is accessed and shared globally.
    Link: https://example.com/world_wide_web_invention
    
    **Instructions for Generating the Markdown Summary:**
    
    1.  **Parse Input Carefully:** Thoroughly read and understand all the provided research entries.
    2.  **Synthesize into a Narrative:** Do not simply list the summaries. Instead, weave them into a flowing and coherent narrative.
        * Focus specifically on the **actions, discoveries, inventions, or key findings** of the authors. What did they *do*?
        * If there are thematic connections or a progression of research evident across different entries, try to highlight these relationships.
    3.  **Integrate Links Naturally:** For each significant contribution or point you discuss from an entry, you must seamlessly incorporate the corresponding HTML link using markdown syntax. The link should provide a way for the reader to get more information about that specific point.
        * For example: `Dr. Ada Lovelace and Charles Babbage conceptualized the Analytical Engine, a precursor to modern computing ([explore their concepts](https://example.com/analytical_engine_details)).`
        * Another example: `The invention of the World Wide Web by Dr. Tim Berners-Lee, which included HTTP and HTML, revolutionized global information sharing ([learn more about this invention](https://example.com/world_wide_web_invention)).`
    4.  **Output Requirements:**
        * The entire output must be a **single string of raw Markdown**.
        * **Crucially, do not include any introductory phrases** like "Here is the summary:", "Certainly, I can help with that:", or any similar conversational text.
        * Similarly, **do not include any concluding remarks** or text outside of the Markdown content itself.
        * Ensure all links provided in the input are used correctly as functional Markdown links in the Markdown output.
    
    **Based on the input data that will be provided immediately following this line, generate the raw Markdown summary now and *only* return the HTML summary.**
    
    {df_string}"""

def pass_to_llm(prompt):
    ip = "141.44.141.132"
    url = f"http://{ip}:80/llama/generate"
    token = "1pAQTJaxP6XxnuxIvm7lIV1QpuwVVcml3R9LDwVtbTwa3"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    

    ollama_llm = OllamaLLM(model="qwen3:32b")
    response = ollama_llm.invoke(prompt)
    
    return response

def get_llm_response(prompt, max_retries=5):
    for attempt in range(max_retries):
        response = pass_to_llm(prompt)
        if response is not None:
            return response
        print(f"Attempt {attempt + 1} failed, retrying...")
        
    print("All retries failed. Continuing with None.")
    return None

In [16]:
from tqdm.notebook import tqdm

In [17]:
max_len = len(data_df.groupby("primary_cat"))

In [18]:
md_responses = []
i = 0
for group, group_data in data_df.groupby("primary_cat"):
    print(f"{i} of {max_len}")
    i+=1
    print(group)
    group_df_string = ""
    for index, row in group_data.iterrows():
        group_df_string += f"""
        Authors: {row["authors"]}
        Summary: {row["summaries"]}
        Link: {row["links"]}
        ---
        
        """
    prompt = get_prompt(group_df_string)
    llm_response = pass_to_llm(prompt)
    if llm_response is None: continue
    md_response = {
        "cat": group,
        "resp": llm_response
    }
    md_responses.append(md_response)

  0%|          | 0/111 [00:00<?, ?it/s]

0 of 111
astro-ph.CO
1 of 111
astro-ph.EP
2 of 111
astro-ph.GA
3 of 111
astro-ph.HE
4 of 111
astro-ph.IM
5 of 111
astro-ph.SR
6 of 111
cond-mat.mes-hall
7 of 111
cond-mat.mtrl-sci
8 of 111
cond-mat.quant-gas
9 of 111
cond-mat.soft
10 of 111
cond-mat.stat-mech
11 of 111
cond-mat.str-el
12 of 111
cond-mat.supr-con
13 of 111
cs.AI
14 of 111
cs.AR
15 of 111
cs.CC
16 of 111
cs.CE
17 of 111
cs.CG
18 of 111
cs.CL
19 of 111
cs.CR
20 of 111
cs.CV
21 of 111
cs.CY
22 of 111
cs.DB
23 of 111
cs.DC
24 of 111
cs.DM
25 of 111
cs.DS
26 of 111
cs.ET
27 of 111
cs.FL
28 of 111
cs.GR
29 of 111
cs.GT
30 of 111
cs.HC
31 of 111
cs.IR
32 of 111
cs.IT
33 of 111
cs.LG
34 of 111
cs.LO
35 of 111
cs.MA
36 of 111
cs.NE
37 of 111
cs.NI
38 of 111
cs.OS
39 of 111
cs.RO
40 of 111
cs.SC
41 of 111
cs.SD
42 of 111
cs.SE
43 of 111
cs.SI
44 of 111
econ.EM
45 of 111
econ.GN
46 of 111
econ.TH
47 of 111
eess.AS
48 of 111
eess.IV
49 of 111
eess.SP
50 of 111
eess.SY
51 of 111
gr-qc
52 of 111
hep-ex
53 of 111
hep-lat
54 of 111
hep

In [19]:
start_date, end_date = get_dates()
final_markdown = f"# Summaries between {start_date} and {end_date}"
for md_resp in md_responses:
    final_markdown += f"""
    ## {md_resp["cat"]}
    {md_resp["resp"]}
    """    

In [20]:
os.makedirs("markdown", exist_ok = True)
with open(f"markdown/{start_date}.md", "w", encoding="utf-8") as f:
    f.write(final_markdown)

In [22]:
final_markdown

'# Summaries between 202505150000 and 202505170000\n    ## astro-ph.CO\n    <think>\nOkay, let\'s see. The user provided a bunch of research papers in cosmology and related fields. They want me to analyze them and generate a detailed summary. Hmm, first, I need to go through each paper and understand their main contributions. Let me start with the first one.\n\nThe first paper by You et al. is about using bio-inspired metaheuristic algorithms for cosmological parameter estimation. They compared algorithms like the Philippine Eagle Optimization Algorithm (PEOA) with genetic algorithms and MCMC. The key point here is that PEOA performed comparably to MCMC in terms of accuracy and precision. Interesting, so nature-inspired algorithms could be viable alternatives in cosmological analysis. I should note that they used mock data and bootstrapping for confidence regions.\n\nNext, the paper by W. d\'Assignies Doumerg et al. is about clustering redshifts for Euclid\'s tomographic bins. They use

In [28]:
import requests
import json

def send_email_via_api(subject: str, content: str, api_token: str = "xDc1hfNfyjgaJHnu", api_url: str = "https://daniel-walke.com/mail/send") -> tuple[int, dict]:
    """
    Sends an email by making a POST request to the specified API endpoint.

    Args:
        subject: The subject of the email.
        content: The HTML or Markdown content of the email.
        api_token: The Bearer token for API authorization.
        api_url: The URL of the API endpoint for sending mail.
                 Defaults to "https://daniel-walke.com/mail/send".

    Returns:
        A tuple containing the HTTP status code and the JSON response from the API.
        If the request fails at the network level or JSON decoding fails,
        it might raise an exception (e.g., requests.exceptions.RequestException, json.JSONDecodeError).
    """
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    payload = {
        "subject": subject,
        "content": content,
    }

    try:
        # Make the POST request
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))

        # Raise an exception for bad status codes (4xx or 5xx)
        # response.raise_for_status() # Optional: uncomment to raise HTTPError for bad responses

        # Try to parse the JSON response
        try:
            response_json = response.json()
        except json.JSONDecodeError:
            # If response is not JSON, return the text content
            response_json = {"error": "Failed to decode JSON", "text_response": response.text}

        return response.status_code, response_json

    except requests.exceptions.RequestException as e:
        # Handle network errors or other request-related issues
        print(f"An error occurred during the request: {e}")
        return 0, {"error": str(e)} # Return 0 or a specific error code for network issues


In [29]:
send_email_via_api(f"Summary {start_date}", final_markdown)

An error occurred during the request: HTTPSConnectionPool(host='daniel-walke.com', port=443): Max retries exceeded with url: /mail/send (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f7cc3f60be0>: Failed to establish a new connection: Socket error: Connection closed unexpectedly')))


(0,
 {'error': "HTTPSConnectionPool(host='daniel-walke.com', port=443): Max retries exceeded with url: /mail/send (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f7cc3f60be0>: Failed to establish a new connection: Socket error: Connection closed unexpectedly')))"})

In [31]:
send_email_via_api(f"Summary {start_date}", "Jo hey was geht")


KeyboardInterrupt



In [30]:
len(final_markdown)

596986

In [40]:
with open("markdown/202505150000.md", 'r', encoding='utf-8') as f:
    content = f.read()
content

'# Summaries between 202505150000 and 202505170000\n    ## astro-ph.CO\n    <think>\nOkay, let\'s see. The user provided a bunch of research papers in cosmology and related fields. They want me to analyze them and generate a detailed summary. Hmm, first, I need to go through each paper and understand their main contributions. Let me start with the first one.\n\nThe first paper by You et al. is about using bio-inspired metaheuristic algorithms for cosmological parameter estimation. They compared algorithms like the Philippine Eagle Optimization Algorithm (PEOA) with genetic algorithms and MCMC. The key point here is that PEOA performed comparably to MCMC in terms of accuracy and precision. Interesting, so nature-inspired algorithms could be viable alternatives in cosmological analysis. I should note that they used mock data and bootstrapping for confidence regions.\n\nNext, the paper by W. d\'Assignies Doumerg et al. is about clustering redshifts for Euclid\'s tomographic bins. They use

In [33]:
import requests
import json

def send_email_via_api(subject: str, content: str, api_token: str = "xDc1hfNfyjgaJHnu", api_url: str = "https://daniel-walke.com/mail/send", bypass_proxy: bool = True) -> tuple[int, dict]:
    """
    Sends an email by making a POST request to the specified API endpoint.

    Args:
        subject: The subject of the email.
        content: The HTML or Markdown content of the email.
        api_token: The Bearer token for API authorization.
        api_url: The URL of the API endpoint for sending mail.
                 Defaults to "https://daniel-walke.com/mail/send".
        bypass_proxy: If True, attempts to bypass system-defined proxies for this request.
                      Set to False to use system proxies if configured.

    Returns:
        A tuple containing the HTTP status code and the JSON response from the API.
        If the request fails at the network level or JSON decoding fails,
        it might raise an exception (e.g., requests.exceptions.RequestException, json.JSONDecodeError).
    """
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    payload = {
        "subject": subject,
        "content": content,
    }

    # Proxy handling
    proxies_to_use = None
    if bypass_proxy:
        proxies_to_use = {
            "http": "http://141.44.122.190:3128/",
            "https": "http://141.44.122.190:3128/",
        }
        print("Attempting to bypass system proxies for this request.")
    else:
        print("Using system-defined proxies (if any) for this request.")


    try:
        # Make the POST request
        # Added proxies=proxies_to_use
        print(proxies_to_use)
        response = requests.post(api_url, headers=headers, data=json.dumps(payload), proxies=proxies_to_use)

        # Raise an exception for bad status codes (4xx or 5xx)
        # response.raise_for_status() # Optional: uncomment to raise HTTPError for bad responses

        # Try to parse the JSON response
        try:
            response_json = response.json()
        except json.JSONDecodeError:
            # If response is not JSON, return the text content
            response_json = {"error": "Failed to decode JSON", "text_response": response.text}

        return response.status_code, response_json

    except requests.exceptions.RequestException as e:
        # Handle network errors or other request-related issues
        print(f"An error occurred during the request: {e}")
        return 0, {"error": str(e)} # Return 0 or a specific error code for network issues

if __name__ == "__main__":
    # --- Configuration - Replace with your actual values ---
    # It's recommended to get the token from a secure source like an environment variable
    # or a configuration file, not hardcoded.
    MY_API_TOKEN = "xDc1hfNfyjgaJHnu" #"YOUR_ACTUAL_API_TOKEN" # <--- IMPORTANT: Replace this with your real API token
    email_subject = "Domain Email Wichtig - Python Test (Proxy Bypass)"
    # For testing, let's use a much shorter message first to isolate the connection issue
    # email_content = "<h1>Hello from My Python Script!</h1><p>This is my test message sent via Python.</p>"
    email_content = "Short test message." # <--- Using a very short message for now
    
    custom_api_url = "https://daniel-walke.com/mail/send" # Or use the default

    # Test with a very long message (approx 600k chars) - uncomment to test after short message works
    # long_email_content = "A" * 596986
    # email_content = long_email_content


    if MY_API_TOKEN == "YOUR_ACTUAL_API_TOKEN":
        print("Please replace 'YOUR_ACTUAL_API_TOKEN' with your actual API token in the script.")
    else:
        print(f"Sending email to: {custom_api_url}")
        print(f"Subject: {email_subject}")
        print(f"Content length: {len(email_content)} characters")
        if len(email_content) < 200: # Only print short content
            print(f"Content: {email_content}")
        else:
            print(f"Content: (Too long to print fully, starts with) {email_content[:100]}...")


        # --- Test with proxy bypass (default) ---
        print("\n--- Testing with proxy bypass ---")
        status_code, response_data = send_email_via_api(
            subject=email_subject,
            content=email_content, # Start with short content
            api_token=MY_API_TOKEN,
            api_url=custom_api_url,
            bypass_proxy=True # Explicitly True
        )

        print(f"\n--- Response (with proxy bypass) ---")
        print(f"Status Code: {status_code}")
        print(f"Response JSON: {response_data}")

        if status_code == 200 and response_data.get("status") == "success":
            print("\nEmail sent successfully via API (with proxy bypass)!")
        elif status_code == 0:
            print("\nFailed to send email due to a request exception (e.g., network error) (with proxy bypass).")
        else:
            print("\nFailed to send email (with proxy bypass). Check status code and response for details.")

        # --- Optional: Test without proxy bypass (to see if system proxy works/fails) ---
        # print("\n--- Testing WITHOUT proxy bypass (using system proxies if any) ---")
        # status_code_no_bypass, response_data_no_bypass = send_email_via_api(
        #     subject=email_subject,
        #     content=email_content, # Start with short content
        #     api_token=MY_API_TOKEN,
        #     api_url=custom_api_url,
        #     bypass_proxy=False
        # )
        # print(f"\n--- Response (WITHOUT proxy bypass) ---")
        # print(f"Status Code: {status_code_no_bypass}")
        # print(f"Response JSON: {response_data_no_bypass}")
        # if status_code_no_bypass == 200 and response_data_no_bypass.get("status") == "success":
        #     print("\nEmail sent successfully via API (using system proxies)!")



Sending email to: https://daniel-walke.com/mail/send
Subject: Domain Email Wichtig - Python Test (Proxy Bypass)
Content length: 19 characters
Content: Short test message.

--- Testing with proxy bypass ---
Attempting to bypass system proxies for this request.
{'http': 'http://141.44.122.190:3128/', 'https': 'http://141.44.122.190:3128/'}



KeyboardInterrupt



In [38]:
proxies = {
  "http": "http://141.44.122.190:3128",
  "https": "http://141.44.122.190:3128",
}


#requests.get("https://daniel-walke.com/mail/health", proxies = proxies)

In [39]:
requests.get("http://httpbin.org/ip", proxies= proxies)

ProxyError: HTTPConnectionPool(host='141.44.122.190', port=3128): Max retries exceeded with url: http://httpbin.org/ip (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7cc6760b20>: Failed to establish a new connection: Socket error: Connection closed unexpectedly')))